<a href="https://colab.research.google.com/github/psm-optimizes/CG-MIR-Cut-Generation/blob/main/Cut_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing Packages**

In [ ]:
pip install coinor.cuppy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install coinor.grumpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install graphviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install py-gnuplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pypolyhedron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Dependencies**

In [ ]:
import numpy as np
import itertools
from coinor.cuppy import *
from cylp.cy import CyClpSimplex
from cylp.py.modeling import CyLPModel, CyLPArray
from coinor.cuppy.cuttingPlanes import *
from coinor.cuppy.milpInstance import MILPInstance
from coinor.cuppy.genericSeparation import GenericSeparation

import graphviz as gviz
import matplotlib as mpb
import matplotlib.pyplot as plt
from coinor.grumpy.polyhedron2D import Polyhedron2D, Figure

In [ ]:
np.set_printoptions(edgeitems=30, linewidth=100000)

In [ ]:
EPS = 5

The following block of code is originally work of *Andrey Lovyagin* and is obtained from https://github.com/r4ndompuff/polyhedral_set. The original code's purpose is to extract the extreme points of a polyhedron. I changed the definition of **extreme_points** function to also return the list of adjacent faces which produce the extreme points. Furthermore, I altered the definition of **check_extreme** function to be compatible with CyClpSimplex package and to use MILP instance's sense (instead of it's non-user-friendly original sense input).
This function helps if someone wants to extract any binding combination of constraints.

This was formerly needed for CGCut, but after discussion with you I decided to stick with the CG multipliers from the tableau to define the CGCut function. However, I decided to leave it here, as part of my effort to find the binding constraints (at optimality) as my initial strategy. In the draft after CGCut or in ZHCGCut (*not useable as a function*) function, you can see how this block was used. In the previous version of CGCut, I only generated u randomly instrad of choosing between 0 and 1/2.

In [ ]:
import numpy as np
import itertools as it
from math import factorial
import re

def permutation(numCons, numVars):
    return factorial(numVars) / (factorial(numVars - numCons) * factorial(numCons))

def matrix_combinations(matr, numVars):
    timed = list(map(list, it.combinations(matr, numVars)))
    return np.array(list(timed))

def array_combinations(arr, numVars):
    timed = list(map(list, it.combinations(arr, numVars)))
    return np.array(list(timed))

def check_extreme(matr, arr, x, numCons):
    for i in range(int(numCons)):
        td_answer = sum(matr[i] * x)
        if m.sense == '>=':
            if td_answer < arr[i]:
                return 0
        elif m.sense == '<=':
            if td_answer > arr[i]:
                return 0
        elif m.sense == '=':
            if td_answer != arr[i]:
                return 0
        else:
            return 0
    return 1

def extreme_points(A, b):
    # Input
    A = np.array(A)
    b = np.array(b)
    numCons, numVars = A.shape
    # Process
    ans_comb = np.zeros((1, numVars))
    arr_comb = array_combinations(b, numVars)
    matr_comb = matrix_combinations(A, numVars)
    for i in range(int(permutation(numVars, numCons))):
        if np.linalg.det(matr_comb[i]) != 0:
            x = np.linalg.solve(np.array(matr_comb[i], dtype='float'),
                                np.array(arr_comb[i], dtype='float'))
            ans_comb = np.vstack([ans_comb, x])
    ans_comb = np.delete(ans_comb, 0, axis=0)
    j = 0
    for i in range(len(ans_comb)):
        if check_extreme(A, b, ans_comb[j], numCons):
            ans_comb = ans_comb
            j += 1
        else:
            ans_comb = np.delete(ans_comb, j, axis=0)
            matr_comb = np.delete(matr_comb, j, axis=0)
    # Output
    return (ans_comb, matr_comb)

# **Chvátal-Gomory Rounding Procedure**

In [ ]:
def CGCut(m, rowInds = None, eps = EPS, debug_print = False):
  '''Returns the inequalities obrtained by Chvátal-Gomory rounding procedure
     based on multipliers extracted from the tableau (inverse of the basis).
  '''

  cuts = []
  lp = m.lp

  for row in range(lp.nConstraints):
    u = np.around(lp.tableau[row, lp.nVariables:], decimals = eps) - np.floor(np.around(lp.tableau[row, lp.nVariables:], decimals = eps))
    lhs = np.matmul(u, m.A)
    pi = np.zeros(len(lhs))
    pi0 = 0
    for i in range(len(lhs)):
      if np.linalg.norm(lhs[i] - np.rint(lhs[i])) <= 10**(-2):
        pi[i] = np.rint(lhs[i])
      else:
        pi[i] = np.floor(lhs[i])

    rhs = np.matmul(u, m.b)
    if np.linalg.norm(rhs - np.rint(rhs)) <= 10**(-2):
      pi0 = np.rint(rhs)
    else:
      pi0 = np.floor(rhs)

    if m.sense == '<=':
        cuts.append((pi, pi0))
    else:
        cuts.append((-pi, -pi0))
  return cuts, []

In [ ]:
'''
Previous Version of CGCut
'''
# cuts = []
# lp = m.lp
# numCons, numVars = m.A.shape

# xtreme_points, adj_ineq = extreme_points(m.A,m.b)
# optind = np.where(xtreme_points == lp.primalVariableSolution['x'])
# A_bind = adj_ineq[optind[0][0]]

# u = np.round(np.random.rand(lp.nConstraints), 2)
# print(u)
# #1
# lhs = np.zeros(len(integerIndices))
# for i in integerIndices:
#   lhs += u[i] * A_bind[i]
# print(lhs)
# #1
# rhs = 0
# for i in integerIndices:
#   rhs = rhs + (u[i] * m.b[np.where(m.A == A_bind[i])[0][0]])
# print(rhs)
# #1

# if (np.dot(lhs, lp.primalVariableSolution['x']) -rhs) > 10**(-4) :
#   print('Invalid inequality (before rounding) for the LP polyhedron')
#   while (np.matmul(lp.primalVariableSolution['x'],lhs) <= rhs):
#     u = np.round(np.random.rand(lp.nConstraints), 2)
#     #2
#     lhs = np.zeros(len(integerIndices))
#     for i in integerIndices:
#       lhs += u[i] * A_bind[i]
#     #2
#     rhs = 0
#     for i in integerIndices:
#       rhs = rhs + (u[i] * lp.rhs[np.where(m.A == A_bind[i])[0][0]])
#     #2

# pi = np.zeros(len(lhs))
# pi0 = 0
# for i in range(len(lhs)):
#   if np.linalg.norm(lhs[i] - np.rint(lhs[i])) <= 10**(-2):
#     pi[i] = np.rint(lhs[i])
#   else:
#     pi[i] = np.floor(lhs[i])

# if np.linalg.norm(rhs - np.rint(rhs)) <= 10**(-2):
#   pi0 = np.rint(rhs)
# else:
#   pi0 = np.floor(rhs)

# if m.sense == '<=':
#     cuts.append((pi, pi0))
# else:
#     cuts.append((-pi, -pi0))

 **Zero-half Cut**

The following function is **not useable**, and is left here as a demonstration of how zero-half cuts were computed. I got the results reported by doing the procedure for only one iteration, without calling it as function by CuPPy or elsewhere. The purpose was only to see how good are these types of cuts.

In [ ]:
def ZHCGCut(m, rowInds = None, eps = EPS, debug_print = False):
  '''Return the inequalities obrtained by Chvátal-Gomory rounding procedure
     using {0, 0.5} multipliers
  '''

  zerohalf = [0, 1/2]
  cuts = []
  lp = m.lp
  numCons, numVars = m.A.shape

  xtreme_points, adj_ineq = extreme_points(m.A,m.b)
  optind = np.where(xtreme_points == lp.primalVariableSolution['x'])
  A_bind = adj_ineq[optind[0][0]]
  # print(A_bind)

  u  = np.random.choice(zerohalf, lp.nConstraints)
  # print('  u = ',  u)
  lhs = np.zeros(len(integerIndices))
  for i in integerIndices:
    lhs += u[i] * A_bind[i]
  # print('lhs = ',lhs)
  rhs = 0
  for i in integerIndices:
    rhs = rhs + (u[i] * m.b[np.where(m.A == A_bind[i])[0][0]])
  # print('rhs = ',rhs)

  # Check if the the inequality (before rounding) supports the LP polyhedron
  if (np.dot(lhs, lp.primalVariableSolution['x']) -rhs) > 10**(-4) :
    print('Invalid inequality (before rounding) for the LP polyhedron')

  pi = np.zeros(len(lhs))
  pi0 = 0
  for i in range(len(lhs)):
    if np.linalg.norm(lhs[i] - np.rint(lhs[i])) <= 10**(-2):
      pi[i] = np.rint(lhs[i])
    else:
      pi[i] = np.floor(lhs[i])

  if np.linalg.norm(rhs - np.rint(rhs)) <= 10**(-2):
    pi0 = np.rint(rhs)
  else:
    pi0 = np.floor(rhs)
  # print('(pi, pi0) = ',pi, pi0)

  return cuts, []

In [ ]:
# pi       = np.array([np.floor(lp.coefMatrix[row,j]) + (np.max(0,(f[j] - f0))/(1 - f0)) for j in integerIndices])
# pi_cont  = np.array([lp.coefMatrix[row,j]/(1-f0) if  lp.coefMatrix[row,j] < 0 else 0 for j in np.delete(range(lp.nVariables), IntInd)])
# pi       = np.zeros(lp.nVariables)
# pi_cont  = np.zeros(lp.nVariables)
# for j in integerIndices:
#   pi[j]  = np.floor(lp.tableau[row,j]) + (np.maximum(0,(f[j] - f0))/(1 - f0))

# pi_cont  = np.array([A[row,j]/(1-f0) if  A[row,j] < 0 else 0 for j in np.delete(range(lp.nVariables), integerIndices)])
# for j in np.delete(range(lp.nVariables), integerIndices):
#     if lp.tableau[row,j] < 0:
#       pi_cont[j]   = np.array(lp.tableau[[row,j]/(1-f0)])
#     else:
#       pi_cont[j]   = 0

# **Mixed Interger Rounding**

Mixed Integer Rounding (MIR) inequality is as follows:
\begin{equation*}
\sum_{j=1}^{p} (\lfloor a_j \rfloor + \frac{(f_j - f_0)^+}{1-f_0}) x_j + \frac{1}{1-f_0} \sum_{p+1 \leq j \leq n : a_j <0} a_j x_j \leq \lfloor b \rfloor.
\end{equation*}
where $f_j = a_j - \lfloor a_j \rfloor$ and $f_0 = b - \lfloor b \rfloor$.







In [ ]:
def MIRCut(m, rowInds = None, eps = EPS, debug_print = False):
    '''Returns the Mixed Integer Rounding (MIR) Inequalities'''

    cuts = []
    lp = m.lp
    sol = lp.primalVariableSolution['x']
    if rowInds is None:
            rowInds = list(range(lp.nConstraints))
    for row in rowInds:
            basicVarInd = lp.basicVariables[row]
            if (basicVarInd in m.integerIndices) and (not isInt(sol[basicVarInd], eps)):
                f0 = getFraction(sol[basicVarInd], eps)
                f = []
                for i in range(lp.nVariables):
                    if i in lp.basicVariables:
                        f.append(0)
                    else:
                      f.append(getFraction(lp.tableau[row, i], eps))

                pi = np.array([np.floor(lp.tableau[row,j]) + (np.maximum(0,f[j] - f0))/(1 - f0) for j in m.integerIndices])
                pi_slacks = np.array([a/(1-f0) if a < 0 else 0 for a in lp.tableau[row, lp.nVariables:]])
                pi -= pi_slacks * lp.coefMatrix
                pi0 = (np.floor(sol[basicVarInd]) - np.dot(pi_slacks, lp.constraintsUpper) if m.sense == '<='
                      else np.floor(sol[basicVarInd]) + np.dot(pi_slacks, lp.constraintsUpper))
                # print('VarInd = ', basicVarInd, ', Cut = ', pi,'<=', pi0)

                if m.sense == '<=':
                    cuts.append((pi, pi0))
                else:
                    cuts.append((-pi, -pi0))

    return cuts, []

# **Tests**

In [ ]:
'''
Problem 1 of the report
'''
numVars = 2
numCons = 5
A = np.array([[-8, 30],
              [-14, 8],
              [10, 10],
              [-1,  0],
              [0,  -1]])

b = np.array([115,
                1,
              127,
                0,
                0,
                 ])

c = np.array([ 0, 1])

sense = ('Max', '<=')
integerIndices = [0, 1]

m = MILPInstance(A = A, b = b, c = c,
                     sense = sense, integerIndices = integerIndices,
                     numVars = numVars)

In [ ]:
'''
Problem 2 of the report
'''
numVars = 2
numCons = 5
A = np.array([[4,  1],
              [1,  4],
              [1, -1],
              [-1, 0],
              [0, -1]])

b = np.array([28,
              27,
               1,
               0,
               0])

c = np.array([2, 5])
sense = ('Max', '<=')
integerIndices = [0, 1]
opt = np.array([17/3, 16/3])

m = MILPInstance(A = A, b = b, c = c,
                     sense = sense, integerIndices = integerIndices,
                     numVars = numVars)

The following two blocks of code was intended to generate (first block) visualize (second block) the cuts obtained by different types of rounding (CR, RU, and LURD).

In [ ]:
'''
Generating cuts obtained by different types of rounding (CR, RU, and LURD).
'''
zerohalf = [0, 1/2]
cuts = []
lp = m.lp
numCons, numVars = m.A.shape

xtreme_points, adj_ineq = extreme_points(m.A,m.b)
optind = np.where(xtreme_points == [7, 5.7])
A_bind = adj_ineq[optind[0][0]]
print(A_bind)

u = np.round(np.random.rand(lp.nConstraints), 2)
# u  = np.random.choice(zerohalf, lp.nConstraints)
print('  u = ',  u)
#1
lhs = np.zeros(len(integerIndices))
for i in integerIndices:
  lhs += u[i] * A_bind[i]
print('lhs = ',lhs)
#1
rhs = 0
for i in integerIndices:
  rhs = rhs + (u[i] * m.b[np.where(m.A == A_bind[i])[0][0]])
print('rhs = ',rhs)
#1

if (np.dot(lhs, lp.primalVariableSolution['x']) -rhs) > 10**(-4) :
  print('Invalid inequality (before rounding) for the LP polyhedron')

pi = np.zeros(len(lhs))
pi0 = 0
for i in range(len(lhs)):
  if np.linalg.norm(lhs[i] - np.rint(lhs[i])) <= 10**(-2):
    pi[i] = np.rint(lhs[i])
  else:
    pi[i] = np.floor(lhs[i])

if np.linalg.norm(rhs - np.rint(rhs)) <= 10**(-2):
  pi0 = np.rint(rhs)
else:
  pi0 = np.floor(rhs)

print('(pi, pi0) = ',pi, pi0)

In [ ]:
'''
Visualizing cuts obtained by different types of rounding (CR, RU, and LURD).
'''
A = np.array([[-8, 30],
              [-14, 8],
              [10, 10],
              [-1,  0],
              [0,  -1]
              ])

b = np.array([115,
                1,
              127,
               0,
               0
              ])
f = Figure()
p = Polyhedron2D(A = A, b = b)
f.add_polyhedron(p, color = 'blue', linestyle = 'solid')
f.set_xlim((-1,16))
f.set_ylim((-1,11))
pI = p.make_integer_hull()
f.add_polyhedron(pI, color = 'red', linestyle = 'dashed', show_int_points = True)
# for i in range(3):
#   u = np.random.rand(5)
#   while np.matmul(np.floor(u[0]*A[0] + u[2]*A[2]),opt) <= np.floor(u[0]*b[0] + u[2]*b[2]):
#     u = np.random.rand(5)
#   f.add_line(np.floor(u[0]*A[0] + u[2]*A[2]), np.floor(u[0]*b[0] + u[2]*b[2]), p.xlim, p.ylim, color = 'black')
# f.add_line(lhs, rhs, p.xlim, p.ylim, color = 'k', linestyle = 'dotted')
# f.add_line(np.floor(lhs), np.floor(rhs), p.xlim, p.ylim, color = 'r', linestyle = 'dotted')   # Rounded down
# f.add_line(np.ceil(lhs), np.ceil(rhs), p.xlim, p.ylim, color = 'g', linestyle = 'dotted')     # Rounded up
# f.add_line(np.ceil(lhs), np.floor(rhs), p.xlim, p.ylim, color = 'c', linestyle = 'dotted')    # Coefs. Up, rhs. Down
# f.add_line(pi, pi0, p.xlim, p.ylim, color = 'm', linestyle = 'dotted')                      # Conservative rounding
f.show(wait_for_click = False)

In [ ]:
A = np.array([[4,  1],
              [1,  4],
              [1, -1],
              [-1, 0],
              [0, -1]])

b = np.array([28,
              27,
               1,
               0,
               0])
f = Figure()
p = Polyhedron2D(A = A, b = b)
f.add_polyhedron(p, color = 'blue', linestyle = 'solid')
f.set_xlim((-1,16))
f.set_ylim((-1, 10))
pI = p.make_integer_hull()
f.add_polyhedron(pI, color = 'red', linestyle = 'dashed', show_int_points = True)
# for i in range(3):
#   u = np.random.rand(5)
#   while np.matmul(np.floor(u[0]*A[0] + u[2]*A[2]),opt) <= np.floor(u[0]*b[0] + u[2]*b[2]):
#     u = np.random.rand(5)
#   f.add_line(np.floor(u[0]*A[0] + u[2]*A[2]), np.floor(u[0]*b[0] + u[2]*b[2]), p.xlim, p.ylim, color = 'black')
# f.add_line(lhs, rhs, p.xlim, p.ylim, color = 'k', linestyle = 'dotted')
# f.add_line(np.floor(lhs), np.floor(rhs), p.xlim, p.ylim, color = 'r', linestyle = 'dotted') # Rounded down
# f.add_line(np.ceil(lhs), np.ceil(rhs), p.xlim, p.ylim, color = 'g', linestyle = 'dotted')   # Rounded up
# f.add_line(np.ceil(lhs), np.floor(rhs), p.xlim, p.ylim, color = 'c', linestyle = 'dotted')  # Coefs. Up, rhs. Down
# f.add_line(pi, pi0, p.xlim, p.ylim, color = 'm', linestyle = 'dotted')                      # Conservative rounding
f.show(wait_for_click = False)

In [ ]:
solve(m, whichCuts = [(CGCut, {})], display = False, debug_print = False, max_iter=10000, max_cuts=10000)

In [ ]:
solve(m, whichCuts = [(MIRCut, {})], display = False, debug_print = False, max_iter=10000, max_cuts=10000)

In [ ]:
solve(m, whichCuts = [(gomoryMixedIntegerCut, {})], display = False, debug_print = False, max_iter=10000, max_cuts=10000)

In [ ]:
solve(MILPInstance(module_name = 'coinor.cuppy.examples.MIP6'),
      whichCuts = [(CGCut, {})],
      display = False, debug_print = False, max_iter=10000, max_cuts=10000)

In [ ]:
solve(MILPInstance(module_name = 'coinor.cuppy.examples.MIP6'),
      whichCuts = [(MIRCut, {})],
      display = False, debug_print = False, max_iter=10000, max_cuts=10000)

In [ ]:
solve(MILPInstance(module_name = 'coinor.cuppy.examples.MIP6'),
      whichCuts = [(gomoryMixedIntegerCut, {})],
      display = False, debug_print = False, max_iter=10000, max_cuts=10000)